In [118]:
import csv
import requests
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
from pathlib import Path

import datalad.api as api
from datalad.api import Dataset, create, copy_file, remove, clone, get
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform

studies = ["CCNP", "BHRC", "NKI", "HBN", "PNC"]
git_repos = [f"https://github.com/ReproBrainChart/{s}_CPAC.git" for s in studies]

### Study Parameters ###
default_dict = dict(
    task="rest",
    run=None,
    acq=None,
    atlas="CC200",
    space="MNI152NLin6ASym",
    reg="36Parameter",
)
study_parameters = {study: default_dict.copy() for study in studies}
study_parameters["PNC"]["acq"] = "singleband"
study_parameters["NKI"]["acq"] = "645"

for s in ["CCNP", "BHRC", "HBN"]:
    if s == "CCNP":
        run = "01"
    else:
        run = "1"
    study_parameters[s]["run"] = run

### Acquisition Times in ms ###
acquisition_times = dict(
    CCNP = 2.5,
    BHRC = 2.000,
    NKI = .645,
    HBN = dict(SI = 1.450, CBIC = .800, RU=.800),
    PNC = 3.000
)

In [102]:
for study in studies:
    out_path = Path(f"./{study}")
    if not out_path.exists():
        out_path.mkdir(exist_ok=True)

    url = f"https://raw.githubusercontent.com/ReproBrainChart/{study}_BIDS/main/study-{study}_desc-participants.tsv"
    response = requests.get(url)
    reader = csv.reader(response.text.splitlines(), skipinitialspace=True)

    with open(out_path / f"{study}_desc-participants.tsv", "w") as f:
        w = csv.writer(f)
        w.writerows(reader)

In [104]:
for study, study_parameter in study_parameters.items():
    df = pd.read_csv(f"{study}/{study}_desc-participants.tsv", delimiter="\t")
    df = df[~df["p_factor_mcelroy_harmonized_all_samples"].isnull()]
    print(len(df))

    glob_str = "*".join(
        [f"{k}-{v}" for k, v in study_parameter.items() if v is not None]
    )
    glob_str = "**/*" + glob_str + "*Mean_timeseries.1D"

    p = Path(f"./{study}_CPAC/cpac_RBCv0")
    files = []
    for _, row in df.iterrows():
        sub_path = p / f"sub-{row.participant_id}/ses-{row.session_id}"
        files.append(list(sub_path.glob(glob_str)))

    print(f"{study}: {len(files)}")

178
CCNP: 178
835
BHRC: 835
474
NKI: 474
2383
HBN: 2383
1600
PNC: 1600


In [130]:
def compute_dynamic_connectome(fpath, window_length=60, acquisition_tr=None):
    """
    Parameters
    ----------
    window_length : int
        Length of window to compute dynamic connectome in seconds. Default=45.
    acquisition_tr : int
        TR in seconds
    """
    assert acquisition_tr is not None
    
    df = pd.read_csv(fpath)
    tr_per_window = int(np.round(window_length / acquisition_tr))

    corrs = []
    for i in range(len(df) - tr_per_window):
        corr = np.corrcoef(df[i:i+tr_per_window], rowvar=False)
        idx = np.triu_indices_from(corr)
        corrs.append(corr[idx])

    return np.array(corrs)

In [131]:
fpath = "./HBN_CPAC/cpac_RBCv0/sub-NDARAJ366ZFA/ses-HBNsiteSI/func/sub-NDARAJ366ZFA_ses-HBNsiteSI_task-rest_atlas-CC200_space-MNI152NLin6ASym_reg-36Parameter_desc-Mean_timeseries.1D"

arr = compute_dynamic_connectome(fpath, acquisition_tr= 1.45)

In [132]:
arr.shape

(377, 20100)

In [133]:
np.save("./test", arr)

In [108]:
a = pd.read_csv("./HBN_CPAC/cpac_RBCv0/sub-NDARAJ366ZFA/ses-HBNsiteSI/func/sub-NDARAJ366ZFA_ses-HBNsiteSI_task-rest_atlas-CC200_space-MNI152NLin6ASym_reg-36Parameter_desc-Mean_timeseries.1D")

In [109]:
a

,#Mean_1,Mean_2,Mean_3,Mean_4,Mean_5,Mean_6,Mean_7,Mean_8,Mean_9,Mean_10,...,Mean_191,Mean_192,Mean_193,Mean_194,Mean_195,Mean_196,Mean_197,Mean_198,Mean_199,Mean_200
0,1.108130,0.078439,0.825072,0.352711,0.178320,-0.388518,-0.937631,-0.272998,0.076236,-0.121087,...,-0.735592,-0.100091,-0.919516,-0.496505,0.793358,0.054195,-0.042998,-0.243878,0.227731,0.017486
1,1.658788,0.227011,1.217894,0.260311,0.038713,-0.365178,-1.049963,-0.595401,0.182205,-0.260332,...,-0.951751,0.052871,-1.432314,-0.364348,1.602766,-0.042868,-0.082862,-0.164676,0.612674,0.009253
2,2.048799,0.410037,1.549874,0.125948,0.076263,-0.301547,-1.051169,-0.883842,0.278445,-0.404225,...,-0.993061,0.245722,-1.775799,-0.098576,2.432679,-0.167204,-0.252889,0.051954,0.976168,-0.045712
3,2.177482,0.611174,1.765459,-0.015215,0.319494,-0.224179,-0.946219,-1.058984,0.331745,-0.518243,...,-0.844262,0.428442,-1.892018,0.253541,3.098134,-0.295201,-0.560451,0.360408,1.224336,-0.154219
4,2.011565,0.807451,1.829011,-0.123422,0.725348,-0.163966,-0.755588,-1.065256,0.321242,-0.567610,...,-0.536073,0.554248,-1.792915,0.615922,3.438072,-0.404365,-0.970918,0.675433,1.284323,-0.311665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1.166412,-1.445156,1.583653,-0.536085,1.830165,0.382866,-0.543367,1.351161,0.541920,-0.512464,...,-1.241168,0.855499,-1.508559,-0.340524,2.484158,0.103048,0.254447,-0.274938,-0.838038,-0.197962
414,1.785793,-1.198727,1.813315,-0.590457,1.982839,0.386944,-0.393208,1.545084,0.583974,-0.558589,...,-1.145720,0.977266,-1.521181,-0.111269,2.839638,0.247252,-0.164566,-0.236857,-0.760931,-0.282399
415,2.157608,-0.857123,1.847345,-0.603061,2.087424,0.339646,-0.261860,1.592940,0.556770,-0.518231,...,-0.857617,0.996652,-1.384777,0.101314,2.954321,0.261084,-0.596671,-0.132237,-0.692354,-0.371741
416,2.204707,-0.497031,1.674887,-0.551910,2.077736,0.249777,-0.204564,1.483686,0.470284,-0.406814,...,-0.467270,0.904743,-1.149978,0.257265,2.794681,0.164697,-0.927685,0.010598,-0.626465,-0.414960


In [119]:
squareform(np.corrcoef(a[:45], rowvar=False))

ValueError: Distance matrix 'X' must be symmetric.

In [124]:
idx, jdx = np.triu_indices_from(a)

In [126]:
idx

array([  0,   0,   0, ..., 198, 198, 199])

In [127]:
a

,#Mean_1,Mean_2,Mean_3,Mean_4,Mean_5,Mean_6,Mean_7,Mean_8,Mean_9,Mean_10,...,Mean_191,Mean_192,Mean_193,Mean_194,Mean_195,Mean_196,Mean_197,Mean_198,Mean_199,Mean_200
0,1.108130,0.078439,0.825072,0.352711,0.178320,-0.388518,-0.937631,-0.272998,0.076236,-0.121087,...,-0.735592,-0.100091,-0.919516,-0.496505,0.793358,0.054195,-0.042998,-0.243878,0.227731,0.017486
1,1.658788,0.227011,1.217894,0.260311,0.038713,-0.365178,-1.049963,-0.595401,0.182205,-0.260332,...,-0.951751,0.052871,-1.432314,-0.364348,1.602766,-0.042868,-0.082862,-0.164676,0.612674,0.009253
2,2.048799,0.410037,1.549874,0.125948,0.076263,-0.301547,-1.051169,-0.883842,0.278445,-0.404225,...,-0.993061,0.245722,-1.775799,-0.098576,2.432679,-0.167204,-0.252889,0.051954,0.976168,-0.045712
3,2.177482,0.611174,1.765459,-0.015215,0.319494,-0.224179,-0.946219,-1.058984,0.331745,-0.518243,...,-0.844262,0.428442,-1.892018,0.253541,3.098134,-0.295201,-0.560451,0.360408,1.224336,-0.154219
4,2.011565,0.807451,1.829011,-0.123422,0.725348,-0.163966,-0.755588,-1.065256,0.321242,-0.567610,...,-0.536073,0.554248,-1.792915,0.615922,3.438072,-0.404365,-0.970918,0.675433,1.284323,-0.311665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1.166412,-1.445156,1.583653,-0.536085,1.830165,0.382866,-0.543367,1.351161,0.541920,-0.512464,...,-1.241168,0.855499,-1.508559,-0.340524,2.484158,0.103048,0.254447,-0.274938,-0.838038,-0.197962
414,1.785793,-1.198727,1.813315,-0.590457,1.982839,0.386944,-0.393208,1.545084,0.583974,-0.558589,...,-1.145720,0.977266,-1.521181,-0.111269,2.839638,0.247252,-0.164566,-0.236857,-0.760931,-0.282399
415,2.157608,-0.857123,1.847345,-0.603061,2.087424,0.339646,-0.261860,1.592940,0.556770,-0.518231,...,-0.857617,0.996652,-1.384777,0.101314,2.954321,0.261084,-0.596671,-0.132237,-0.692354,-0.371741
416,2.204707,-0.497031,1.674887,-0.551910,2.077736,0.249777,-0.204564,1.483686,0.470284,-0.406814,...,-0.467270,0.904743,-1.149978,0.257265,2.794681,0.164697,-0.927685,0.010598,-0.626465,-0.414960


In [125]:
a[idx, jdx]

InvalidIndexError: (array([  0,   0,   0, ..., 198, 198, 199]), array([  0,   1,   2, ..., 198, 199, 199]))